## 安装 Apache TVM

## 步骤1: 检查系统环境

确认当前目录为`~/workspace`

In [1]:
!pwd

/home/ec2-user/SageMaker/workspace


此外，我们需要确认nvcc的版本需要和nvidia-smi中CUDA的驱动版本一致。

In [2]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Sat_Aug_25_21:08:01_CDT_2018
Cuda compilation tools, release 10.0, V10.0.130
Tue May 26 03:35:23 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   22C    P0    22W / 300W |      0MiB / 16160MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+

In [5]:
!sudo rm /usr/local/cuda
!sudo ln -s /usr/local/cuda-10.2 /usr/local/cuda
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_19:24:38_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


## 步骤2: 安装编译工具

这里，我们通过操作系统的安装包管理器，安装编译TVM所需的工具

 * TVM需要支持C++14的C++编译器。这里我们用一下命令安装g++-7.2
 * CMake（3.0以上版本）
 * LLVM (4.0以上版本)

In [ ]:
##### Uncomment the following lines if you are trying to install TVM in SageMaker Studio ####
# !apt update
# !apt install -y libcudnn7-dev libnvinfer-dev cuda-nvcc-10-2 cuda-nvrtc-10-2 cuda-nvrtc-dev-10-2 
# !apt install -y cmake llvm-6.0-dev

#### Uncomment the following lines if you are trying to install TVM in SageMaker Notebook ####
!sudo yum update -y
!sudo yum install -y gcc72 gcc72-c++ cmake3 llvm6.0-devel
!gcc --version
!g++ --version

Loaded plugins: dkms-build-requires, priorities, update-motd, upgrade-helper,
              : versionlock
No packages marked for update


## 步骤3: 下载最新版本的Apache TVM

Apache TVM 的官方Github地址是 http://github.com/apache/incubator-tvm ，为了提升下载速度可以从 https://gitee.com/mirrors/tvm 下载 Apache TVM.

In [7]:
!git clone --recursive https://gitee.com/mirrors/tvm.git tvm_upstream    
!cd tvm_upstream && git fetch origin master && git merge origin/master && cd ..

fatal: destination path 'tvm_upstream' already exists and is not an empty directory.
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 78 (delta 68), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (78/78), done.
From https://gitee.com/mirrors/tvm
 * branch                master     -> FETCH_HEAD
   0833b07b4..cc79591f5  master     -> origin/master
Updating 0833b07b4..cc79591f5
Fast-forward
 include/tvm/relay/attrs/algorithm.h                |   5 +-
 include/tvm/relay/attrs/transform.h                |   2 +-
 include/tvm/runtime/ndarray.h                      |   6 +-
 python/tvm/relay/_parser.py                        |   2 +
 python/tvm/relay/op/_algorithm.py                  |  68 +++++++++
 python/tvm/relay/op/_tensor.py                     |  41 +++---
 python/tvm/relay/op/_tensor_grad.py                |   8 +-
 python/tvm/relay/op/_transform.py                  |   2 

In [8]:
!mkdir -p tvm_upstream/build
!cp config.cmake tvm_upstream/build
!cd tvm_upstream/build && cmake -DCMAKE_BUILD_TYPE=Release .. && make -j4

-- Build with RPC support...
-- Build with Graph runtime support...
-- VTA build with VTA_HW_PATH=/home/ec2-user/SageMaker/workspace/tvm_upstream/3rdparty/vta-hw
-- Build VTA runtime with target: sim
-- Found CUDA_TOOLKIT_ROOT_DIR=/usr/local/cuda-10.1
-- Found CUDA_CUDA_LIBRARY=/usr/local/cuda-10.1/targets/x86_64-linux/lib/stubs/libcuda.so
-- Found CUDA_CUDART_LIBRARY=/usr/local/cuda-10.1/lib64/libcudart.so
-- Found CUDA_NVRTC_LIBRARY=/usr/local/cuda-10.1/lib64/libnvrtc.so
-- Found CUDA_CUDNN_LIBRARY=/usr/local/cuda-10.1/lib64/libcudnn.so
-- Found CUDA_CUBLAS_LIBRARY=/usr/lib64/libcublas.so
-- Found CUDA_CUBLASLT_LIBRARY=CUDA_CUBLASLT_LIBRARY-NOTFOUND
-- Build with CUDA support
-- Build with cuDNN support
-- Use llvm-config=/usr/bin/llvm-config-6.0-64
-- /usr/lib64/llvm6.0/include
-- Found LLVM_INCLUDE_DIRS=/usr/lib64/llvm6.0/include
-- Found LLVM_DEFINITIONS= -DNDEBUG -D_GNU_SOURCE -D__STDC_CONSTANT_MACROS -D__STDC_FORMAT_MACROS -D__STDC_LIMIT_MACROS
-- Found TVM_LLVM_VERSION=60
-- Bu

In [9]:
!cd tvm_upstream && make cython3

cd python; python3 setup.py build_ext --inplace
running build_ext
building 'tvm._ffi._cy3.core' extension
gcc -pthread -B /home/ec2-user/anaconda3/envs/mxnet_p36/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I../include/ -I../3rdparty/dmlc-core/include -I../3rdparty/dlpack/include -I/home/ec2-user/anaconda3/envs/mxnet_p36/include/python3.6m -c tvm/_ffi/_cython/core.cpp -o build/temp.linux-x86_64-3.6/tvm/_ffi/_cython/core.o -std=c++14
gcc: error: unrecognized command line option ‘-std=c++14’
error: command 'gcc' failed with exit status 1
make: *** [cython3] Error 1


In [31]:
!pip3 install --index-url https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple/ --user decorator numpy scipy attrs

Looking in indexes: https://mirrors.tuna.tsinghua.edu.cn/pypi/web/simple/
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [17]:
import sys
sys.path.insert(0, "tvm_upstream/python")
sys.path.insert(0, "tvm_upstream/topi/python")

import tvm
import topi

In [18]:
!PYTHONPATH=`pwd`/tvm_upstream/python:`pwd`/tvm_upstream/topi/python python3 tvm_upstream/tutorials/optimize/opt_conv_cuda.py

Convolution: 90.204707 ms


In [29]:
!mkdir -p ~/.tvm/tophub
!cp cuda_v0.08.log ~/.tvm/tophub
!cp llvm_v0.04.log ~/.tvm/tophub

!mkdir -p ~/.tvm_test_data/data
!cp cat.png ~/.tvm_test_data/data
!cp imagenet1000_clsid_to_human.txt ~/.tvm_test_data/data

In [32]:
!PYTHONPATH=`pwd`/tvm_upstream/python:`pwd`/tvm_upstream/topi/python python3 tvm_upstream/tutorials/frontend/from_mxnet.py

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/distributed/config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})
File /home/ec2-user/.tvm_test_data/data/cat.png exists, skip.
File /home/ec2-user/.tvm_test_data/data/imagenet1000_clsid_to_human.txt exists, skip.
Figure(640x480)
x (1, 3, 224, 224)
Cannot find config for target=cuda, workload=('conv2d_nchw.cuda', ('TENSOR', (1, 64, 56, 56), 'float32'), ('TENSOR', (64, 64, 3, 3), 'float32'), (1, 1), (1, 1, 1, 1), (1, 1), 'float32'). A fallback configuration is used, which may bring great performance regression.
Cannot find config for target=cuda, workload=('conv2d_nchw.cuda', ('TENSOR', (1, 128, 28, 28), 'float32'), ('TENSOR', (128, 128, 3, 3), 'float32'), (1, 1), (1, 1, 1, 1), (1, 1), 'float32'). A fallback configuration is used, which may bring great performan